In [1]:
import pandas as pd
import numpy as np

artist_data = pd.read_csv('datasets/ydata-ymusic-user-artist-rating.csv')
artist_data = artist_data.loc[artist_data['rating'] <= 100]
artist_data.sort_values('rating', ascending=False)

,user_id,artist_id,rating
1048574,17484,1010722,100
263778,4346,1043477,100
446869,7519,1006971,100
100111,1675,1012152,100
446870,7519,1007098,100
...,...,...,...
839286,14081,1004599,0
608438,10439,1024496,0
839284,14081,1004561,0
839283,14081,1004546,0


In [2]:
#preparing data table for analysis
user_rating = pd.pivot_table(data=artist_data, values='rating', index='user_id', columns='artist_id')
user_rating.head()

artist_id,24538,1000004,1000006,1000010,1000012,1000015,1000016,1000018,1000021,1000023,...,1101383,1101399,1101400,1101401,1101416,1101471,1101613,1101630,1101671,1101719
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#calculating mean for every artist
avg_rating = pd.DataFrame(artist_data.groupby('artist_id')['rating'].mean().reset_index())
avg_rating.columns
avg_rating

,artist_id,rating
0,24538,90.000000
1,1000004,44.731183
2,1000006,56.207692
3,1000010,52.000000
4,1000012,43.884956
...,...,...
14142,1101471,5.555556
14143,1101613,90.000000
14144,1101630,7.454545
14145,1101671,68.292683


In [4]:
avg_rating.sort_values('rating', ascending = False).head(20)

,artist_id,rating
11853,1069861,100.0
11316,1054044,100.0
8168,1031121,100.0
2520,1009432,100.0
11317,1054050,100.0
4716,1017988,100.0
12814,1098250,100.0
2527,1009455,100.0
8158,1031090,100.0
9372,1038802,100.0


In [5]:
#artist_id = 1005698
#see artist 1058037 rated by user
artist = 1058037
artist_user_rating = user_rating[artist]
artist_user_rating.head()

user_id
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: 1058037, dtype: float64

In [6]:
#applying correlation for the entire user rating dataframe
#Collaborative Filtering Song Recommendation
#Pearson Correlation
recommended_artist = user_rating.corrwith(artist_user_rating)
recommended_artist

/Users/admin/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/admin/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


artist_id
24538     NaN
1000004   NaN
1000006   NaN
1000010   NaN
1000012   NaN
           ..
1101471   NaN
1101613   NaN
1101630   NaN
1101671   NaN
1101719   NaN
Length: 14147, dtype: float64

In [7]:
#removing nan values
recommended_artist_corr = pd.DataFrame(recommended_artist, columns=['Correlation'])
recommended_artist_corr.dropna(inplace=True)

In [8]:
recommended_artist_corr.reset_index()

,artist_id,Correlation
0,1000264,-1.0
1,1000362,-1.0
2,1001077,-1.0
3,1002815,-1.0
4,1003380,-1.0
...,...,...
72,1058037,1.0
73,1059036,1.0
74,1083194,-1.0
75,1098033,1.0


In [13]:
#sort the recommended artist correlation
recommended_artist_corr = recommended_artist_corr.sort_values('Correlation', ascending=False).reset_index()
recommended_artist_corr

,artist_id,Correlation
0,1098306,1.0
1,1098033,1.0
2,1019053,1.0
3,1059036,1.0
4,1017855,1.0
...,...,...
72,1021992,-1.0
73,1001077,-1.0
74,1002815,-1.0
75,1003380,-1.0


In [22]:
print(type(recommended_artist_corr.head(5)))
artist_list = recommended_artist_corr.head(5)['artist_id'].tolist()
artist_list

<class 'pandas.core.frame.DataFrame'>


[1098306, 1098033, 1019053, 1059036, 1017855]

In [ ]:
# TODO:
#filtering still need to be done
# print artist names
# use same concept for user/song/rating
# refactor and cleanup

In [16]:
artist_name_data = pd.read_csv('datasets/ydata-ymusic-artist-names.csv')

In [17]:

for artist in artist_list:
    artist_name_data['artist_id'].where(artist == ['artist_id'].values()):
        

,artist_id,artist
0,-100,Not Applicable
1,-99,Unknown Artist
2,1000001,"Bobby ""O"""
3,1000002,"Jimmy ""Z"""
4,1000003,'68 Comeback
...,...,...
97951,1101110,14 Karat Soul
97952,1101111,The Relativez
97953,1101112,Crooked I
97954,1101113,SKG


In [33]:
artist_names = artist_name_data.where(artist_name_data['artist_id'].isin(artist_list))

In [34]:
artist_names

,artist_id,artist
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
97951,NaN,NaN
97952,NaN,NaN
97953,NaN,NaN
97954,NaN,NaN
